# NP Semantic Segmentation

Let's import all the relevant classes

In [1]:
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
os.getcwd()

'/Users/ayaccobi/ai-lab-nlp'

In [7]:
import examples

In [2]:
from examples.np_semantic_segmentation.data import NpSemanticSegData
from examples.np_semantic_segmentation.preprocess_tratz2011 import *
from examples.np_semantic_segmentation.data import *
from nlp_architect.models.np_semantic_segmentation import NpSemanticSegClassifier

Using TensorFlow backend.


## Preparing the data

The first step is the download the dataset into a folder. You can download Tratz 2011 et al. dataset from the following link: [Tratz 2011 Dataset](https://vered1986.github.io/papers/Tratz2011_Dataset.tar.gz). Is also available in [here](https://www.isi.edu/publications/licensed-sw/fanseparser/index.html). (The terms and conditions of the data set license apply. Intel does not grant any rights to the data files or database).

After downloading and unzipping the dataset, the following method will labels some portion of the data, and will output two `.csv` files that will assist us to train and evaluate the trained model.

In [3]:
dataset_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation'
preprocess_tratz_2011(dataset_path)

CSV file is saved in /Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/tratz2011_coarse_grained_random/train.csv
CSV file is saved in /Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/tratz2011_coarse_grained_random/val.csv


Once the dataset is saved and labeled we need to vectories the data:

In [4]:
# labeled_data_path is the output of preprocess_tratz_2011()
labeled_train_data_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/tratz2011_coarse_grained_random/train.csv'
labeled_val_data_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/tratz2011_coarse_grained_random/val.csv'
word2vec_path = '/Users/ayaccobi/workspace/word_embeddings/GoogleNews-vectors-negative300.bin.gz'
# output_path is location to save the vectors
train_output_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/prepared_data_train.csv'
val_output_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/prepared_data_val.csv'
http_proxy = None
https_proxy = None

In [ ]:

prepare_data(labeled_train_data_path, train_output_path, word2vec_path, http_proxy, https_proxy)
prepare_data(labeled_val_data_path, val_output_path, word2vec_path, http_proxy, https_proxy)

Start loading NLTK Collocation scoring (this might take a while...)
Start loading Word2Vec model (this might take a while...)


We now need to load the data into NpSemanticSegmentation object.

In [17]:
data_set = NpSemanticSegData(train_output_path, train_to_test_ratio=0.8)

## Build the model

In [22]:
    model_file_path = 'np_semantic_segmentation.h5'
    num_epochs = 200
    model = NpSemanticSegClassifier(num_epochs=200, callback_args=None)
    input_dim = data_set.train_set_x.shape[1]
    model.build(input_dim)
    model.fit(data_set.train_set)


Epoch 1/200
 - 0s - loss: 1.1516 - binary_accuracy: 0.6977 - precision_score: 0.2126 - recall_score: 0.1663 - f1: 0.1866
Epoch 2/200
 - 0s - loss: 1.0157 - binary_accuracy: 0.7558 - precision_score: 0.2481 - recall_score: 0.0807 - f1: 0.1215
Epoch 3/200
 - 0s - loss: 0.8626 - binary_accuracy: 0.7946 - precision_score: 0.7442 - recall_score: 0.0846 - f1: 0.1424
Epoch 4/200
 - 0s - loss: 1.0398 - binary_accuracy: 0.7907 - precision_score: 0.4961 - recall_score: 0.0191 - f1: 0.0367
Epoch 5/200
 - 0s - loss: 1.0024 - binary_accuracy: 0.7907 - precision_score: 0.0000e+00 - recall_score: 0.0000e+00 - f1: 0.0000e+00
Epoch 6/200
 - 0s - loss: 0.9380 - binary_accuracy: 0.7907 - precision_score: 0.4961 - recall_score: 0.0372 - f1: 0.0692
Epoch 7/200
 - 0s - loss: 1.0358 - binary_accuracy: 0.7868 - precision_score: 0.4134 - recall_score: 0.0368 - f1: 0.0674
Epoch 8/200
 - 0s - loss: 0.9215 - binary_accuracy: 0.7907 - precision_score: 0.0000e+00 - recall_score: 0.0000e+00 - f1: 0.0000e+00
Epoch 9/

Epoch 68/200
 - 0s - loss: 0.7160 - binary_accuracy: 0.9302 - precision_score: 0.9425 - recall_score: 0.7028 - f1: 0.8045
Epoch 69/200
 - 0s - loss: 0.7505 - binary_accuracy: 0.9186 - precision_score: 0.9383 - recall_score: 0.6554 - f1: 0.7696
Epoch 70/200
 - 0s - loss: 0.7990 - binary_accuracy: 0.9225 - precision_score: 0.9249 - recall_score: 0.6685 - f1: 0.7713
Epoch 71/200
 - 0s - loss: 0.6200 - binary_accuracy: 0.9341 - precision_score: 0.9647 - recall_score: 0.7076 - f1: 0.8119
Epoch 72/200
 - 0s - loss: 0.6708 - binary_accuracy: 0.9264 - precision_score: 0.9922 - recall_score: 0.6426 - f1: 0.7800
Epoch 73/200
 - 0s - loss: 0.6822 - binary_accuracy: 0.9341 - precision_score: 0.9450 - recall_score: 0.7166 - f1: 0.8150
Epoch 74/200
 - 0s - loss: 0.7917 - binary_accuracy: 0.9302 - precision_score: 0.9415 - recall_score: 0.6951 - f1: 0.7972
Epoch 75/200
 - 0s - loss: 0.6559 - binary_accuracy: 0.9225 - precision_score: 0.9686 - recall_score: 0.6408 - f1: 0.7708
Epoch 76/200
 - 0s - los

Epoch 135/200
 - 0s - loss: 0.5404 - binary_accuracy: 0.9651 - precision_score: 0.9674 - recall_score: 0.8419 - f1: 0.9003
Epoch 136/200
 - 0s - loss: 0.5594 - binary_accuracy: 0.9651 - precision_score: 0.9922 - recall_score: 0.8316 - f1: 0.9046
Epoch 137/200
 - 0s - loss: 0.5534 - binary_accuracy: 0.9651 - precision_score: 0.9922 - recall_score: 0.8269 - f1: 0.9020
Epoch 138/200
 - 0s - loss: 0.5850 - binary_accuracy: 0.9651 - precision_score: 1.0000 - recall_score: 0.8299 - f1: 0.9067
Epoch 139/200
 - 0s - loss: 0.5897 - binary_accuracy: 0.9651 - precision_score: 0.9922 - recall_score: 0.8393 - f1: 0.9054
Epoch 140/200
 - 0s - loss: 0.5397 - binary_accuracy: 0.9612 - precision_score: 0.9578 - recall_score: 0.8560 - f1: 0.9028
Epoch 141/200
 - 0s - loss: 0.6522 - binary_accuracy: 0.9612 - precision_score: 0.9674 - recall_score: 0.8259 - f1: 0.8910
Epoch 142/200
 - 0s - loss: 0.5237 - binary_accuracy: 0.9535 - precision_score: 1.0000 - recall_score: 0.7621 - f1: 0.8626
Epoch 143/200
 -

Great! We now have a MLP classifier for collocations. Let's evaluate it on the val_data_set:

In [23]:
val_dataset = data_set = NpSemanticSegData(val_output_path, train_to_test_ratio=1)

loss, binary_accuracy, precision, recall, f1 = model.eval(val_dataset.train_set)
print('loss = %.1f%%' % (loss))
print('Test binary_accuracy rate = %.1f%%' % (binary_accuracy * 100))
print('Test precision rate = %.1f%%' % (precision * 100))
print('Test recall rate = %.1f%%' % (recall * 100))
print('Test f1 rate = %.1f%%' % (f1 * 100))


Saved model to disk
loss = 0.6%
Test binary_accuracy rate = 84.6%
Test precision rate = 75.0%
Test recall rate = 42.9%
Test f1 rate = 54.5%
